In [1]:
import sys
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

# 현재 노트북 파일의 경로를 기준으로 프로젝트 루트 경로를 계산
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..'))

# sys.path에 프로젝트 루트 경로가 없으면 추가
if project_root not in sys.path:
  sys.path.append(project_root)


In [2]:
from src.data.db_handler import DBHandler

db_handler = DBHandler(db_name="data_lake")

df = db_handler.fetch_data(table_name="kr_stock_price_basic")

column_index = df.columns

df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 85 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   id                        200 non-null    int64              
 1   ticker                    200 non-null    object             
 2   iscd_stat_cls_code        200 non-null    object             
 3   marg_rate                 200 non-null    object             
 4   rprs_mrkt_kor_name        200 non-null    object             
 5   new_hgpr_lwpr_cls_code    200 non-null    object             
 6   bstp_kor_isnm             200 non-null    object             
 7   temp_stop_yn              200 non-null    object             
 8   oprc_rang_cont_yn         200 non-null    object             
 9   clpr_rang_cont_yn         200 non-null    object             
 10  crdt_able_yn              200 non-null    object             
 11  grmn_rate_cls_code 

,id,ticker,iscd_stat_cls_code,marg_rate,rprs_mrkt_kor_name,new_hgpr_lwpr_cls_code,bstp_kor_isnm,temp_stop_yn,oprc_rang_cont_yn,clpr_rang_cont_yn,crdt_able_yn,grmn_rate_cls_code,elw_pblc_yn,stck_prpr,prdy_vrss,prdy_vrss_sign,prdy_ctrt,acml_tr_pbmn,acml_vol,prdy_vrss_vol_rate,stck_oprc,stck_hgpr,stck_lwpr,stck_mxpr,stck_llam,stck_sdpr,wghn_avrg_stck_prc,hts_frgn_ehrt,frgn_ntby_qty,pgtr_ntby_qty,pvt_scnd_dmrs_prc,pvt_frst_dmrs_prc,pvt_pont_val,pvt_frst_dmsp_prc,pvt_scnd_dmsp_prc,dmrs_val,dmsp_val,cpfn,rstc_wdth_prc,stck_fcam,stck_sspr,aspr_unit,hts_deal_qty_unit_val,lstn_stcn,hts_avls,per,pbr,stac_month,vol_tnrt,eps,bps,d250_hgpr,d250_hgpr_date,d250_hgpr_vrss_prpr_rate,d250_lwpr,d250_lwpr_date,d250_lwpr_vrss_prpr_rate,stck_dryy_hgpr,dryy_hgpr_vrss_prpr_rate,dryy_hgpr_date,stck_dryy_lwpr,dryy_lwpr_vrss_prpr_rate,dryy_lwpr_date,w52_hgpr,w52_hgpr_vrss_prpr_ctrt,w52_hgpr_date,w52_lwpr,w52_lwpr_vrss_prpr_ctrt,w52_lwpr_date,whol_loan_rmnd_rate,ssts_yn,stck_shrn_iscd,fcam_cnnm,cpfn_cnnm,apprch_rate,frgn_hldn_qty,vi_cls_code,ovtm_vi_cls_code,last_ssts_cntg_qty,invt_caful_yn,mrkt_warn_cls_code,short_over_yn,sltr_yn,mang_issu_cls_code,updated_at
0,1,005930,55,20.00,KOSPI200,신고가,전기·전자,N,N,N,Y,40,Y,79700,-800,5,-0.99,1672547819584,20898386,81.57,81100,81200,79600,104600,56400,80500,80032.73,51.11,-537726,938828,81766,81132,79866,79232,77966,81450,79550,7780,24100,100,61180,100,1,5919637922,4717951,16.10,1.38,12,0.35,4950.00,57930.00,81200,20250919,-1.85,49900,20241114,59.72,81200,-1.85,20250919,50800,56.89,20250203,81200,-1.85,20250919,49900,59.72,20241114,0.17,Y,005930,100,"7,780 억",NaN,3025406868,N,N,489779,N,00,N,N,N,2025-09-21 18:40:39.954205+00:00
1,2,000660,55,20.00,KOSPI200,신고가,전기·전자,N,N,N,Y,40,Y,353000,0,3,0.00,1546050743059,4385543,96.89,351500,360000,348000,458500,247500,353000,352559.79,56.08,-982503,-357729,369333,361166,348833,340666,328333,365250,344750,36577,105500,5000,254160,500,1,728002365,2569848,12.99,3.38,12,0.60,27182.00,104567.00,360000,20250919,-1.94,144700,20240919,143.95,360000,-1.94,20250919,162700,116.96,20250409,360000,-1.94,20250919,144700,143.95,20240919,0.16,Y,000660,"5,000","36,576 억",NaN,408260104,N,N,84763,N,00,N,N,N,2025-09-21 18:40:39.954205+00:00
2,3,373220,55,20.00,KOSPI200,NaN,전기·전자,N,N,N,Y,40,Y,349500,-5000,5,-1.41,64980756750,185906,106.97,351500,354500,347500,460500,248500,354500,349535.95,4.09,-23961,-36176,359500,357000,352000,349500,344500,358250,350750,1170,106000,500,269420,500,1,234000000,817830,-80.27,3.87,12,0.08,-4354.00,90240.00,444000,20241008,-21.28,266000,20250523,31.39,403000,-13.28,20250731,266000,31.39,20250523,444000,-21.28,20241008,266000,31.39,20250523,0.11,Y,373220,500,"1,170 억",NaN,9580313,N,N,45469,N,00,N,N,N,2025-09-21 18:40:39.954205+00:00
3,4,207940,55,20.00,KOSPI200,NaN,제약,N,N,N,Y,40,Y,1024000,-5000,5,-0.49,93367438000,91201,177.89,1023000,1030000,1020000,1337000,721000,1029000,1023779.40,12.86,5913,7932,1037666,1033332,1025666,1021332,1013666,1035500,1023500,1779,308000,2500,802620,1000,1,71174000,728822,67.28,6.68,12,0.13,15221.00,153212.00,1209000,20250214,-15.30,907000,20241118,12.90,1209000,-15.30,20250214,924000,10.82,20250102,1209000,-15.30,20250214,907000,12.90,20241118,0.09,Y,207940,"2,500","1,779 억",NaN,9150802,N,N,1706,N,00,N,N,N,2025-09-21 18:40:39.954205+00:00
4,5,012450,55,20.00,KOSPI200,NaN,운송장비·부품,N,N,N,Y,40,Y,1022000,-1000,5,-0.10,372660454884,363904,173.56,1010000,1041000,1005000,1329000,717000,1023000,1024102.56,44.77,3680,-2295,1077666,1050332,1024666,997332,971666,1037500,984500,2703,306000,5000,654720,1000,1,51563401,526978,22.13,9.44,12,0.71,46183.00,108270.00,1052000,20250918,-2.85,265870,20241210,284.40,1052000,-2.85,20250918,325445,214.03,20250102,1052000,-2.85,20250918,270000,278.52,20241210,0.34,Y,012450,"5,000","2,703 억",NaN,23082752,N,N,5937,N,00,N,N,N,2025-09-21 18:40:39.954205+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [3]:
final_cols = column_index[1:]

In [4]:
df[final_cols] = df[final_cols].replace(r'^\s*$', np.nan, regex=True)
df[final_cols] = df[final_cols].replace(['0', 0], np.nan)

# 결측치 개수 계산
missing_value_counts = df[final_cols].isnull().sum()

# 전체 행의 개수 계산
total_rows = len(df[final_cols])

# 결측치 비율 계산
missing_value_percent = ((missing_value_counts / total_rows) * 100).round(2)


# --- 3. 결과 취합 및 출력 ---
missing_df = pd.DataFrame({
  'missing_count': missing_value_counts,
  'missing_percent': missing_value_percent
})

# 결측치가 많은 순서대로 정렬하여 출력
missing_df = missing_df[missing_df['missing_count'] > 0]
missing_df = missing_df.sort_values(by='missing_count', ascending=False)

missing_df

,missing_count,missing_percent
grmn_rate_cls_code,11,5.5
prdy_vrss,8,4.0
frgn_ntby_qty,1,0.5


In [5]:
missing_cols = missing_df[missing_df['missing_percent'] >= 50].index.to_list()

final_cols = [col for col in final_cols if col not in missing_cols]

print(f"결측 컬럼 : {missing_cols}")

print(f"[원본 컬럼 : {len(column_index)-1}] - [결측 컬럼 : {len(missing_cols)}] = [후보 컬럼 : {len(final_cols)}]")

결측 컬럼 : []
[원본 컬럼 : 84] - [결측 컬럼 : 0] = [후보 컬럼 : 84]


In [6]:
nunique_counts = df[final_cols].nunique()

# 전체 행의 개수 계산
total_rows = len(df[final_cols])

# 결측치 비율 계산
nunique_value_percent = ((nunique_counts / total_rows) * 100).round(2)

unique_cols = nunique_counts[nunique_counts == 1].index.tolist()

# --- 3. 결과 취합 및 출력 ---
nunique_df = pd.DataFrame({
  'nunique_count': nunique_counts,
  'nunique_percent': nunique_value_percent
})

# 결측치가 많은 순서대로 정렬하여 출력
nunique_df = nunique_df[nunique_df['nunique_count'] > 1]
nunique_df = nunique_df.sort_values(by='nunique_count', ascending=False)

nunique_df.T

,ticker,wghn_avrg_stck_prc,frgn_hldn_qty,stck_shrn_iscd,bps,eps,hts_avls,lstn_stcn,pgtr_ntby_qty,acml_tr_pbmn,acml_vol,prdy_vrss_vol_rate,pvt_frst_dmsp_prc,pvt_frst_dmrs_prc,frgn_ntby_qty,last_ssts_cntg_qty,stck_sspr,d250_lwpr_vrss_prpr_rate,pvt_pont_val,pvt_scnd_dmrs_prc,per,w52_lwpr_vrss_prpr_ctrt,stck_hgpr,hts_frgn_ehrt,dryy_lwpr_vrss_prpr_rate,w52_hgpr_vrss_prpr_ctrt,pvt_scnd_dmsp_prc,dmrs_val,dmsp_val,cpfn,stck_sdpr,stck_mxpr,d250_hgpr_vrss_prpr_rate,cpfn_cnnm,stck_dryy_lwpr,d250_lwpr,w52_lwpr,stck_llam,d250_hgpr,dryy_hgpr_vrss_prpr_rate,stck_lwpr,stck_dryy_hgpr,stck_prpr,w52_hgpr,stck_oprc,rstc_wdth_prc,prdy_ctrt,pbr,whol_loan_rmnd_rate,prdy_vrss,d250_hgpr_date,w52_hgpr_date,dryy_hgpr_date,vol_tnrt,d250_lwpr_date,w52_lwpr_date,dryy_lwpr_date,bstp_kor_isnm,marg_rate,fcam_cnnm,stck_fcam,aspr_unit,grmn_rate_cls_code,prdy_vrss_sign,elw_pblc_yn,new_hgpr_lwpr_cls_code,iscd_stat_cls_code,crdt_able_yn
nunique_count,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,199.0,199.0,199.0,199.0,198.0,198.0,198.0,198.0,197.0,197.0,197.0,197.0,197.0,196.0,196.0,196.0,196.0,196.0,195.0,195.0,194.0,193.0,192.0,192.0,192.0,191.0,191.0,190.0,190.0,190.0,190.0,190.0,187.0,178.0,166.0,141.0,122.0,108.0,98.0,95.0,93.0,91.0,54.0,54.0,42.0,21.0,6.0,6.0,6.0,6.0,3.0,3.0,2.0,2.0,2.0,2.0
nunique_percent,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,99.5,99.5,99.5,99.5,99.0,99.0,99.0,99.0,98.5,98.5,98.5,98.5,98.5,98.0,98.0,98.0,98.0,98.0,97.5,97.5,97.0,96.5,96.0,96.0,96.0,95.5,95.5,95.0,95.0,95.0,95.0,95.0,93.5,89.0,83.0,70.5,61.0,54.0,49.0,47.5,46.5,45.5,27.0,27.0,21.0,10.5,3.0,3.0,3.0,3.0,1.5,1.5,1.0,1.0,1.0,1.0


In [7]:
final_cols = [col for col in final_cols if col not in unique_cols]

print(f"유니크 컬럼 : {unique_cols}")

print(f"[원본 컬럼 : {len(column_index)-1}] - [결측 컬럼 : {len(missing_cols)}] - [유니크 컬럼 : {len(unique_cols)}] = [후보 컬럼 : {len(final_cols)}]")

유니크 컬럼 : ['rprs_mrkt_kor_name', 'temp_stop_yn', 'oprc_rang_cont_yn', 'clpr_rang_cont_yn', 'hts_deal_qty_unit_val', 'stac_month', 'ssts_yn', 'apprch_rate', 'vi_cls_code', 'ovtm_vi_cls_code', 'invt_caful_yn', 'mrkt_warn_cls_code', 'short_over_yn', 'sltr_yn', 'mang_issu_cls_code', 'updated_at']
[원본 컬럼 : 84] - [결측 컬럼 : 0] - [유니크 컬럼 : 16] = [후보 컬럼 : 68]


In [8]:
df[final_cols]

,ticker,iscd_stat_cls_code,marg_rate,new_hgpr_lwpr_cls_code,bstp_kor_isnm,crdt_able_yn,grmn_rate_cls_code,elw_pblc_yn,stck_prpr,prdy_vrss,prdy_vrss_sign,prdy_ctrt,acml_tr_pbmn,acml_vol,prdy_vrss_vol_rate,stck_oprc,stck_hgpr,stck_lwpr,stck_mxpr,stck_llam,stck_sdpr,wghn_avrg_stck_prc,hts_frgn_ehrt,frgn_ntby_qty,pgtr_ntby_qty,pvt_scnd_dmrs_prc,pvt_frst_dmrs_prc,pvt_pont_val,pvt_frst_dmsp_prc,pvt_scnd_dmsp_prc,dmrs_val,dmsp_val,cpfn,rstc_wdth_prc,stck_fcam,stck_sspr,aspr_unit,lstn_stcn,hts_avls,per,pbr,vol_tnrt,eps,bps,d250_hgpr,d250_hgpr_date,d250_hgpr_vrss_prpr_rate,d250_lwpr,d250_lwpr_date,d250_lwpr_vrss_prpr_rate,stck_dryy_hgpr,dryy_hgpr_vrss_prpr_rate,dryy_hgpr_date,stck_dryy_lwpr,dryy_lwpr_vrss_prpr_rate,dryy_lwpr_date,w52_hgpr,w52_hgpr_vrss_prpr_ctrt,w52_hgpr_date,w52_lwpr,w52_lwpr_vrss_prpr_ctrt,w52_lwpr_date,whol_loan_rmnd_rate,stck_shrn_iscd,fcam_cnnm,cpfn_cnnm,frgn_hldn_qty,last_ssts_cntg_qty
0,005930,55,20.00,신고가,전기·전자,Y,40,Y,79700,-800,5,-0.99,1672547819584,20898386,81.57,81100,81200,79600,104600,56400,80500,80032.73,51.11,-537726,938828,81766,81132,79866,79232,77966,81450,79550,7780,24100,100,61180,100,5919637922,4717951,16.10,1.38,0.35,4950.00,57930.00,81200,20250919,-1.85,49900,20241114,59.72,81200,-1.85,20250919,50800,56.89,20250203,81200,-1.85,20250919,49900,59.72,20241114,0.17,005930,100,"7,780 억",3025406868,489779
1,000660,55,20.00,신고가,전기·전자,Y,40,Y,353000,NaN,3,0.00,1546050743059,4385543,96.89,351500,360000,348000,458500,247500,353000,352559.79,56.08,-982503,-357729,369333,361166,348833,340666,328333,365250,344750,36577,105500,5000,254160,500,728002365,2569848,12.99,3.38,0.60,27182.00,104567.00,360000,20250919,-1.94,144700,20240919,143.95,360000,-1.94,20250919,162700,116.96,20250409,360000,-1.94,20250919,144700,143.95,20240919,0.16,000660,"5,000","36,576 억",408260104,84763
2,373220,55,20.00,NaN,전기·전자,Y,40,Y,349500,-5000,5,-1.41,64980756750,185906,106.97,351500,354500,347500,460500,248500,354500,349535.95,4.09,-23961,-36176,359500,357000,352000,349500,344500,358250,350750,1170,106000,500,269420,500,234000000,817830,-80.27,3.87,0.08,-4354.00,90240.00,444000,20241008,-21.28,266000,20250523,31.39,403000,-13.28,20250731,266000,31.39,20250523,444000,-21.28,20241008,266000,31.39,20250523,0.11,373220,500,"1,170 억",9580313,45469
3,207940,55,20.00,NaN,제약,Y,40,Y,1024000,-5000,5,-0.49,93367438000,91201,177.89,1023000,1030000,1020000,1337000,721000,1029000,1023779.40,12.86,5913,7932,1037666,1033332,1025666,1021332,1013666,1035500,1023500,1779,308000,2500,802620,1000,71174000,728822,67.28,6.68,0.13,15221.00,153212.00,1209000,20250214,-15.30,907000,20241118,12.90,1209000,-15.30,20250214,924000,10.82,20250102,1209000,-15.30,20250214,907000,12.90,20241118,0.09,207940,"2,500","1,779 억",9150802,1706
4,012450,55,20.00,NaN,운송장비·부품,Y,40,Y,1022000,-1000,5,-0.10,372660454884,363904,173.56,1010000,1041000,1005000,1329000,717000,1023000,1024102.56,44.77,3680,-2295,1077666,1050332,1024666,997332,971666,1037500,984500,2703,306000,5000,654720,1000,51563401,526978,22.13,9.44,0.71,46183.00,108270.00,1052000,20250918,-2.85,265870,20241210,284.40,1052000,-2.85,20250918,325445,214.03,20250102,1052000,-2.85,20250918,270000,278.52,20241210,0.34,012450,"5,000","2,703 억",23082752,5937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,093370,57,100.00,NaN,화학,N,NaN,N,6230,-90,5,-1.42,3763664435,605183,50.02,6380,6390,6160,8210,4430,6320,6219.03,7.33,-21464,-35866,6620,6470,6330,6180,6040,6545,6255,536,1890,500,4550,10,107255330,6682,-9.34,2.05,0.56,-667.00,3044.00,6880,20250915,-9.45,3800,20250409,63.95,6880,-9.45,20250915,3800,63.95,20250409,6880,-9.45,20250915,3800,63.95,20250409,1.97,093370,500,536 억,7863514,8112
196,145720,55,30.00,NaN,의료·정밀기기,Y,40,N,58500,-100,5,-0.17,2245137800,38375,106.79,58300,59000,58000,76100,4110

In [9]:
df[final_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 68 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ticker                    200 non-null    object
 1   iscd_stat_cls_code        200 non-null    object
 2   marg_rate                 200 non-null    object
 3   new_hgpr_lwpr_cls_code    200 non-null    object
 4   bstp_kor_isnm             200 non-null    object
 5   crdt_able_yn              200 non-null    object
 6   grmn_rate_cls_code        189 non-null    object
 7   elw_pblc_yn               200 non-null    object
 8   stck_prpr                 200 non-null    object
 9   prdy_vrss                 192 non-null    object
 10  prdy_vrss_sign            200 non-null    object
 11  prdy_ctrt                 200 non-null    object
 12  acml_tr_pbmn              200 non-null    object
 13  acml_vol                  200 non-null    object
 14  prdy_vrss_vol_rate        

In [10]:
print(final_cols)

['ticker', 'iscd_stat_cls_code', 'marg_rate', 'new_hgpr_lwpr_cls_code', 'bstp_kor_isnm', 'crdt_able_yn', 'grmn_rate_cls_code', 'elw_pblc_yn', 'stck_prpr', 'prdy_vrss', 'prdy_vrss_sign', 'prdy_ctrt', 'acml_tr_pbmn', 'acml_vol', 'prdy_vrss_vol_rate', 'stck_oprc', 'stck_hgpr', 'stck_lwpr', 'stck_mxpr', 'stck_llam', 'stck_sdpr', 'wghn_avrg_stck_prc', 'hts_frgn_ehrt', 'frgn_ntby_qty', 'pgtr_ntby_qty', 'pvt_scnd_dmrs_prc', 'pvt_frst_dmrs_prc', 'pvt_pont_val', 'pvt_frst_dmsp_prc', 'pvt_scnd_dmsp_prc', 'dmrs_val', 'dmsp_val', 'cpfn', 'rstc_wdth_prc', 'stck_fcam', 'stck_sspr', 'aspr_unit', 'lstn_stcn', 'hts_avls', 'per', 'pbr', 'vol_tnrt', 'eps', 'bps', 'd250_hgpr', 'd250_hgpr_date', 'd250_hgpr_vrss_prpr_rate', 'd250_lwpr', 'd250_lwpr_date', 'd250_lwpr_vrss_prpr_rate', 'stck_dryy_hgpr', 'dryy_hgpr_vrss_prpr_rate', 'dryy_hgpr_date', 'stck_dryy_lwpr', 'dryy_lwpr_vrss_prpr_rate', 'dryy_lwpr_date', 'w52_hgpr', 'w52_hgpr_vrss_prpr_ctrt', 'w52_hgpr_date', 'w52_lwpr', 'w52_lwpr_vrss_prpr_ctrt', 'w52_l

| 컬럼명 | 한글컬럼명 | 중요도 | 이유 |
| :--- | :--- | :--- | :--- |
| `ticker` | 종목코드 | **필수** | 데이터의 Primary Key. 모든 정보를 연결하는 기준. |
| `iscd_stat_cls_code` | 종목 상태 구분 코드 | 중 | 거래정지, 관리종목 등 이벤트성 피처로 활용 가능. |
| `marg_rate` | 증거금 비율 | 중 | 종목의 신용 위험도를 나타내는 간접 지표. |
| `new_hgpr_lwpr_cls_code` | 신고가/신저가 구분 | 상 | 시장의 모멘텀을 직접적으로 나타내는 중요한 기술적 지표. |
| `bstp_kor_isnm` | 업종명 | 상 | 산업별 트렌드 분석 및 섹터 로테이션 전략에 필수적인 피처. |
| `crdt_able_yn` | 신용거래 가능여부 | 중 | 종목의 유동성 및 안정성과 관련된 피처. |
| `grmn_rate_cls_code` | 담보대출 등급 코드 | 중 | 신용 위험도를 나타내는 추가적인 간접 지표. |
| `elw_pblc_yn` | ELW 발행 여부 | 중 | 파생상품 존재 여부로, 변동성에 영향을 줄 수 있음. |
| `stck_prpr` | **주가 (종가)** | **필수** | 예측 모델의 핵심 Target 또는 가장 중요한 피처. |
| `prdy_vrss` | 전일 대비 등락 | **필수** | 주가 변동의 절대량을 나타내는 핵심 파생 피처. |
| `prdy_vrss_sign` | 전일 대비 부호 | **필수** | 주가 상승/하락 방향성을 나타내는 핵심 피처. |
| `prdy_ctrt` | 전일 대비 등락률 | **필수** | 정규화된 주가 변동률로, 모델 학습에 직접 사용. |
| `acml_tr_pbmn` | 누적 거래대금 | 상 | 시장의 관심도와 유동성을 나타내는 핵심 지표. |
| `acml_vol` | 누적 거래량 | 상 | 거래대금과 함께 시장 활성도를 판단하는 중요 지표. |
| `prdy_vrss_vol_rate` | 전일 대비 거래량 비율 | 상 | 거래량의 급증/급감을 파악하여 추세 변화를 감지. |
| `stck_oprc` | 시가 | **필수** | 당일 주가 흐름의 시작점으로, 캔들차트의 기본 요소. |
| `stck_hgpr` | 고가 | **필수** | 당일 최고가로, 변동성 및 저항선 분석에 사용. |
| `stck_lwpr` | 저가 | **필수** | 당일 최저가로, 변동성 및 지지선 분석에 사용. |
| `stck_mxpr` | 상한가 | 중 | 가격 제한폭 정보로, 이상 급등 시나리오에 활용. |
| `stck_llam` | 하한가 | 중 | 가격 제한폭 정보로, 이상 급락 시나리오에 활용. |
| `stck_sdpr` | 기준가 | 중 | 가격 변동의 기준이 되는 정보. |
| `wghn_avrg_stck_prc` | 가중평균주가 (VWAP) | 상 | 거래량을 가중치로 한 평균 가격으로, 실제 체결 동향 파악. |
| `hts_frgn_ehrt` | 외국인 소진율 | 상 | 외국인 투자자 지분율로, 수급 분석의 핵심. |
| `frgn_ntby_qty` | 외국인 순매수 수량 | 상 | 외국인 자금 유입/유출을 직접적으로 보여주는 지표. |
| `pgtr_ntby_qty` | 기관 순매수 수량 | 상 | 기관 자금 유입/유출을 직접적으로 보여주는 지표. |
| `pvt_scnd_dmrs_prc` | 피봇 2차 저항선 | 중 | 단기 주가 저항선을 예측하는 기술적 지표. |
| `pvt_frst_dmrs_prc` | 피봇 1차 저항선 | 중 | 단기 주가 저항선을 예측하는 기술적 지표. |
| `pvt_pont_val` | 피봇 포인트 | 중 | 주가의 중심 가격으로, 지지와 저항을 판단하는 기준. |
| `pvt_frst_dmsp_prc` | 피봇 1차 지지선 | 중 | 단기 주가 지지선을 예측하는 기술적 지표. |
| `pvt_scnd_dmsp_prc` | 피봇 2차 지지선 | 중 | 단기 주가 지지선을 예측하는 기술적 지표. |
| `dmrs_val` | 당일 저항선 | 중 | 기술적 분석에 사용되는 저항선 가격. |
| `dmsp_val` | 당일 지지선 | 중 | 기술적 분석에 사용되는 지지선 가격. |
| `cpfn` | 시가총액 | **필수** | 기업의 규모를 나타내는 가장 기본적인 펀더멘탈 지표. |
| `rstc_wdth_prc` | 가격제한폭 | 중 | 일일 최대 변동 가능 범위를 나타내는 정보. |
| `stck_fcam` | 액면가 | 중 | 주식의 초기 설정 가격으로, 직접적 예측력은 낮으나 기본 정보. |
| `stck_sspr` | 결산가 | 중 | 회계 결산 기준 가격. |
| `aspr_unit` | 호가 단위 | 중 | 거래 시스템의 기본 정보. |
| `lstn_stcn` | 상장 주식 수 | 상 | 시가총액과 함께 기업 규모를 판단하는 중요 피처. |
| `hts_avls` | 시가총액 (억) | **필수** | `cpfn`과 동일한 정보. 기업 규모 판단의 핵심. |
| `per` | 주가수익비율 (PER) | 상 | 기업의 가치 평가(Valuation)에 사용되는 핵심 펀더멘탈 지표. |
| `pbr` | 주가순자산비율 (PBR) | 상 | 기업의 가치 평가(Valuation)에 사용되는 핵심 펀더멘탈 지표. |
| `vol_tnrt` | 회전율 | 상 | 주식의 유동성 및 시장 관심도를 나타내는 지표. |
| `eps` | 주당순이익 (EPS) | 상 | 기업의 수익성을 나타내는 핵심 펀더멘탈 지표. |
| `bps` | 주당순자산 (BPS) | 상 | 기업의 재무 안정성을 나타내는 핵심 펀더멘탈 지표. |
| `d250_hgpr` | 250일 최고가 | 상 | 중장기적 주가 모멘텀 및 저항선을 판단하는 지표. |
| `d250_hgpr_date` | 250일 최고가 달성일 | 중 | 최고가 시점 정보를 통해 기간별 모멘텀 분석에 활용. |
| `d250_hgpr_vrss_prpr_rate` | 250일 최고가 대비 등락률 | 상 | 현재 주가가 중장기 고점 대비 어느 위치인지 나타냄. |
| `d250_lwpr` | 250일 최저가 | 상 | 중장기적 주가 모멘텀 및 지지선을 판단하는 지표. |
| `d250_lwpr_date` | 250일 최저가 달성일 | 중 | 최저가 시점 정보를 통해 기간별 모멘텀 분석에 활용. |
| `d250_lwpr_vrss_prpr_rate` | 250일 최저가 대비 등락률 | 상 | 현재 주가가 중장기 저점 대비 어느 위치인지 나타냄. |
| `stck_dryy_hgpr` | 연중 최고가 | 상 | 연간 기준 주가 모멘텀 및 저항선을 판단하는 지표. |
| `dryy_hgpr_vrss_prpr_rate` | 연중 최고가 대비 등락률 | 상 | 현재 주가가 연고점 대비 어느 위치인지 나타냄. |
| `dryy_hgpr_date` | 연중 최고가 달성일 | 중 | 연고점 시점 정보를 통해 기간별 모멘텀 분석에 활용. |
| `stck_dryy_lwpr` | 연중 최저가 | 상 | 연간 기준 주가 모멘텀 및 지지선을 판단하는 지표. |
| `dryy_lwpr_vrss_prpr_rate` | 연중 최저가 대비 등락률 | 상 | 현재 주가가 연저점 대비 어느 위치인지 나타냄. |
| `dryy_lwpr_date` | 연중 최저가 달성일 | 중 | 연저점 시점 정보를 통해 기간별 모멘텀 분석에 활용. |
| `w52_hgpr` | 52주 최고가 | 상 | `d250_hgpr`와 유사, 중장기 모멘텀 판단의 핵심 지표. |
| `w52_hgpr_vrss_prpr_ctrt` | 52주 최고가 대비 등락률 | 상 | 현재 주가가 52주 고점 대비 어느 위치인지 나타냄. |
| `w52_hgpr_date` | 52주 최고가 달성일 | 중 | 52주 고점 시점 정보를 통해 기간별 모멘텀 분석에 활용. |
| `w52_lwpr` | 52주 최저가 | 상 | `d250_lwpr`와 유사, 중장기 지지선 판단의 핵심 지표. |
| `w52_lwpr_vrss_prpr_ctrt` | 52주 최저가 대비 등락률 | 상 | 현재 주가가 52주 저점 대비 어느 위치인지 나타냄. |
| `w52_lwpr_date` | 52주 최저가 달성일 | 중 | 52주 저점 시점 정보를 통해 기간별 모멘텀 분석에 활용. |
| `whol_loan_rmnd_rate` | 대차잔고 비율 | 중 | 공매도 선행지표로 활용될 수 있는 수급 관련 정보. |
| `stck_shrn_iscd` | 단축 종목코드 | **필수** | `ticker`와 동일한 정보. Primary Key. |
| `fcam_cnnm` | 액면가 (문자열) | 중 | `stck_fcam`의 문자열 형태. 전처리 후 사용 가능. |
| `cpfn_cnnm` | 시가총액 (문자열) | 중 | `cpfn`의 문자열 형태. 전처리 후 사용 가능. |
| `frgn_hldn_qty` | 외국인 보유수량 | 상 | `hts_frgn_ehrt`와 함께 외국인 수급을 판단하는 핵심 지표. |
| `last_ssts_cntg_qty` | 최종결제 이행수량 | 중 | 결제 안정성과 관련된 파생 정보. |

In [11]:
final_cols = [
  'ticker',
  'iscd_stat_cls_code',
  'marg_rate',
  'new_hgpr_lwpr_cls_code',
  'bstp_kor_isnm',
  'crdt_able_yn',
  'grmn_rate_cls_code',
  'elw_pblc_yn',
  'stck_prpr',
  'prdy_vrss',
  'prdy_vrss_sign',
  'prdy_ctrt',
  'acml_tr_pbmn',
  'acml_vol',
  'prdy_vrss_vol_rate',
  'stck_oprc',
  'stck_hgpr',
  'stck_lwpr',
  'stck_mxpr',
  'stck_llam',
  'stck_sdpr',
  'wghn_avrg_stck_prc',
  'hts_frgn_ehrt',
  'frgn_ntby_qty',
  'pgtr_ntby_qty',
  'pvt_scnd_dmrs_prc',
  'pvt_frst_dmrs_prc',
  'pvt_pont_val',
  'pvt_frst_dmsp_prc',
  'pvt_scnd_dmsp_prc',
  'dmrs_val',
  'dmsp_val',
  'cpfn',
  'rstc_wdth_prc',
  'stck_fcam',
  'stck_sspr',
  'aspr_unit',
  'lstn_stcn',
  'hts_avls',
  'per',
  'pbr',
  'vol_tnrt',
  'eps',
  'bps',
  'd250_hgpr',
  'd250_hgpr_date',
  'd250_hgpr_vrss_prpr_rate',
  'd250_lwpr',
  'd250_lwpr_date',
  'd250_lwpr_vrss_prpr_rate',
  'stck_dryy_hgpr',
  'dryy_hgpr_vrss_prpr_rate',
  'dryy_hgpr_date',
  'stck_dryy_lwpr',
  'dryy_lwpr_vrss_prpr_rate',
  'dryy_lwpr_date',
  'w52_hgpr',
  'w52_hgpr_vrss_prpr_ctrt',
  'w52_hgpr_date',
  'w52_lwpr',
  'w52_lwpr_vrss_prpr_ctrt',
  'w52_lwpr_date',
  'whol_loan_rmnd_rate',
  'stck_shrn_iscd',
  'fcam_cnnm',
  'cpfn_cnnm',
  'frgn_hldn_qty',
  'last_ssts_cntg_qty'
]

In [12]:
df[final_cols]

,ticker,iscd_stat_cls_code,marg_rate,new_hgpr_lwpr_cls_code,bstp_kor_isnm,crdt_able_yn,grmn_rate_cls_code,elw_pblc_yn,stck_prpr,prdy_vrss,prdy_vrss_sign,prdy_ctrt,acml_tr_pbmn,acml_vol,prdy_vrss_vol_rate,stck_oprc,stck_hgpr,stck_lwpr,stck_mxpr,stck_llam,stck_sdpr,wghn_avrg_stck_prc,hts_frgn_ehrt,frgn_ntby_qty,pgtr_ntby_qty,pvt_scnd_dmrs_prc,pvt_frst_dmrs_prc,pvt_pont_val,pvt_frst_dmsp_prc,pvt_scnd_dmsp_prc,dmrs_val,dmsp_val,cpfn,rstc_wdth_prc,stck_fcam,stck_sspr,aspr_unit,lstn_stcn,hts_avls,per,pbr,vol_tnrt,eps,bps,d250_hgpr,d250_hgpr_date,d250_hgpr_vrss_prpr_rate,d250_lwpr,d250_lwpr_date,d250_lwpr_vrss_prpr_rate,stck_dryy_hgpr,dryy_hgpr_vrss_prpr_rate,dryy_hgpr_date,stck_dryy_lwpr,dryy_lwpr_vrss_prpr_rate,dryy_lwpr_date,w52_hgpr,w52_hgpr_vrss_prpr_ctrt,w52_hgpr_date,w52_lwpr,w52_lwpr_vrss_prpr_ctrt,w52_lwpr_date,whol_loan_rmnd_rate,stck_shrn_iscd,fcam_cnnm,cpfn_cnnm,frgn_hldn_qty,last_ssts_cntg_qty
0,005930,55,20.00,신고가,전기·전자,Y,40,Y,79700,-800,5,-0.99,1672547819584,20898386,81.57,81100,81200,79600,104600,56400,80500,80032.73,51.11,-537726,938828,81766,81132,79866,79232,77966,81450,79550,7780,24100,100,61180,100,5919637922,4717951,16.10,1.38,0.35,4950.00,57930.00,81200,20250919,-1.85,49900,20241114,59.72,81200,-1.85,20250919,50800,56.89,20250203,81200,-1.85,20250919,49900,59.72,20241114,0.17,005930,100,"7,780 억",3025406868,489779
1,000660,55,20.00,신고가,전기·전자,Y,40,Y,353000,NaN,3,0.00,1546050743059,4385543,96.89,351500,360000,348000,458500,247500,353000,352559.79,56.08,-982503,-357729,369333,361166,348833,340666,328333,365250,344750,36577,105500,5000,254160,500,728002365,2569848,12.99,3.38,0.60,27182.00,104567.00,360000,20250919,-1.94,144700,20240919,143.95,360000,-1.94,20250919,162700,116.96,20250409,360000,-1.94,20250919,144700,143.95,20240919,0.16,000660,"5,000","36,576 억",408260104,84763
2,373220,55,20.00,NaN,전기·전자,Y,40,Y,349500,-5000,5,-1.41,64980756750,185906,106.97,351500,354500,347500,460500,248500,354500,349535.95,4.09,-23961,-36176,359500,357000,352000,349500,344500,358250,350750,1170,106000,500,269420,500,234000000,817830,-80.27,3.87,0.08,-4354.00,90240.00,444000,20241008,-21.28,266000,20250523,31.39,403000,-13.28,20250731,266000,31.39,20250523,444000,-21.28,20241008,266000,31.39,20250523,0.11,373220,500,"1,170 억",9580313,45469
3,207940,55,20.00,NaN,제약,Y,40,Y,1024000,-5000,5,-0.49,93367438000,91201,177.89,1023000,1030000,1020000,1337000,721000,1029000,1023779.40,12.86,5913,7932,1037666,1033332,1025666,1021332,1013666,1035500,1023500,1779,308000,2500,802620,1000,71174000,728822,67.28,6.68,0.13,15221.00,153212.00,1209000,20250214,-15.30,907000,20241118,12.90,1209000,-15.30,20250214,924000,10.82,20250102,1209000,-15.30,20250214,907000,12.90,20241118,0.09,207940,"2,500","1,779 억",9150802,1706
4,012450,55,20.00,NaN,운송장비·부품,Y,40,Y,1022000,-1000,5,-0.10,372660454884,363904,173.56,1010000,1041000,1005000,1329000,717000,1023000,1024102.56,44.77,3680,-2295,1077666,1050332,1024666,997332,971666,1037500,984500,2703,306000,5000,654720,1000,51563401,526978,22.13,9.44,0.71,46183.00,108270.00,1052000,20250918,-2.85,265870,20241210,284.40,1052000,-2.85,20250918,325445,214.03,20250102,1052000,-2.85,20250918,270000,278.52,20241210,0.34,012450,"5,000","2,703 억",23082752,5937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,093370,57,100.00,NaN,화학,N,NaN,N,6230,-90,5,-1.42,3763664435,605183,50.02,6380,6390,6160,8210,4430,6320,6219.03,7.33,-21464,-35866,6620,6470,6330,6180,6040,6545,6255,536,1890,500,4550,10,107255330,6682,-9.34,2.05,0.56,-667.00,3044.00,6880,20250915,-9.45,3800,20250409,63.95,6880,-9.45,20250915,3800,63.95,20250409,6880,-9.45,20250915,3800,63.95,20250409,1.97,093370,500,536 억,7863514,8112
196,145720,55,30.00,NaN,의료·정밀기기,Y,40,N,58500,-100,5,-0.17,2245137800,38375,106.79,58300,59000,58000,76100,4110

In [13]:
final_df = df[final_cols].copy()

# --- 1단계: 날짜 타입 자동 변환 ---
remaining_object_cols = final_df.select_dtypes(include='object').columns.tolist()
for col in remaining_object_cols:
  original_valid_count = final_df[col].count()
  if original_valid_count == 0: continue
  
  for format in ['%Y%m%d', '%Y-%m-%d', '%Y%m', '%Y-%m']:
    date_series = pd.to_datetime(final_df[col], format=format,errors='coerce')
    success_rate = date_series.count() / original_valid_count
    
    if success_rate > 0.95: # 95% 임계값
      final_df[col] = date_series
      break

# --- 2단계: 저비율 카테고리 자동 변환 ---
object_cols = final_df.select_dtypes(include='object').columns.tolist()
for col in object_cols:
  ratio = final_df[col].nunique() / len(final_df[col])
  if ratio < 0.05: # 5% 임계값
    final_df[col] = final_df[col].astype('category')


In [14]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 68 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   ticker                    200 non-null    object        
 1   iscd_stat_cls_code        200 non-null    category      
 2   marg_rate                 200 non-null    category      
 3   new_hgpr_lwpr_cls_code    200 non-null    category      
 4   bstp_kor_isnm             200 non-null    object        
 5   crdt_able_yn              200 non-null    category      
 6   grmn_rate_cls_code        189 non-null    category      
 7   elw_pblc_yn               200 non-null    category      
 8   stck_prpr                 200 non-null    object        
 9   prdy_vrss                 192 non-null    object        
 10  prdy_vrss_sign            200 non-null    category      
 11  prdy_ctrt                 200 non-null    object        
 12  acml_tr_pbmn          

| 컬럼명 | 현재 타입 | 적정 타입 | 이유 |
| :--- | :--- | :--- | :--- |
| `ticker` | object | category | 고유 식별자이며 종류가 한정적이므로 category가 효율적. |
| `iscd_stat_cls_code` | category | category | 최적 타입. 코드성 데이터이므로 category 타입 유지. |
| `marg_rate` | category | float32 | 소수점을 포함하는 비율 데이터이므로 float32로 변환 필요. |
| `new_hgpr_lwpr_cls_code` | category | category | 최적 타입. 범주형 데이터이므로 category 타입 유지. |
| `bstp_kor_isnm` | object | category | 업종명은 종류가 제한된 범주형 데이터. |
| `crdt_able_yn` | category | bool | 'Y'/'N' 값을 가지므로 bool 타입이 더 적합. |
| `grmn_rate_cls_code` | category | category | 최적 타입. 코드성 데이터이므로 category 타입 유지. |
| `elw_pblc_yn` | category | bool | 'Y'/'N' 값을 가지므로 bool 타입이 더 적합. |
| `stck_prpr` | object | int32 | 주가 데이터로, 정수형(int32)으로 변환. |
| `prdy_vrss` | object | int32 | 가격 변동폭은 정수형(int32)으로 변환. |
| `prdy_vrss_sign` | category | category | 최적 타입. 코드성 데이터이므로 category 타입 유지. |
| `prdy_ctrt` | object | float32 | 소수점을 포함하는 등락률 데이터. |
| `acml_tr_pbmn` | object | int64 | 값이 매우 클 수 있으므로 int64로 처리. |
| `acml_vol` | object | int64 | 값이 클 수 있으므로 int64로 처리. |
| `prdy_vrss_vol_rate` | object | float32 | 소수점을 포함하는 비율 데이터. |
| `stck_oprc` | object | int32 | 주가 데이터. |
| `stck_hgpr` | object | int32 | 주가 데이터. |
| `stck_lwpr` | object | int32 | 주가 데이터. |
| `stck_mxpr` | object | int32 | 주가 데이터. |
| `stck_llam` | object | int32 | 주가 데이터. |
| `stck_sdpr` | object | int32 | 주가 데이터. |
| `wghn_avrg_stck_prc` | object | float32 | 소수점을 포함하는 평균 가격 데이터. |
| `hts_frgn_ehrt` | object | float32 | 소수점을 포함하는 비율 데이터. |
| `frgn_ntby_qty` | object | int64 | 순매수량은 음수/양수를 포함하며 값이 클 수 있음. |
| `pgtr_ntby_qty` | object | int64 | 순매수량은 음수/양수를 포함하며 값이 클 수 있음. |
| `pvt_scnd_dmrs_prc` | object | int32 | 가격 데이터. |
| `pvt_frst_dmrs_prc` | object | int32 | 가격 데이터. |
| `pvt_pont_val` | object | int32 | 가격 데이터. |
| `pvt_frst_dmsp_prc` | object | int32 | 가격 데이터. |
| `pvt_scnd_dmsp_prc` | object | int32 | 가격 데이터. |
| `dmrs_val` | object | int32 | 가격 데이터. |
| `dmsp_val` | object | int32 | 가격 데이터. |
| `cpfn` | object | int64 | 시가총액은 매우 큰 정수. |
| `rstc_wdth_prc` | object | int32 | 가격 데이터. |
| `stck_fcam` | category | int32 | 액면가는 정수형 데이터이므로 int32로 변환 필요. |
| `stck_sspr` | object | int32 | 가격 데이터. |
| `aspr_unit` | category | int32 | 호가 단위는 정수형 데이터이므로 int32로 변환 필요. |
| `lstn_stcn` | object | int64 | 상장주식수는 매우 큰 정수. |
| `hts_avls` | object | int64 | 시가총액은 매우 큰 정수. |
| `per` | object | float32 | 소수점을 포함하는 비율 데이터. |
| `pbr` | object | float32 | 소수점을 포함하는 비율 데이터. |
| `vol_tnrt` | object | float32 | 소수점을 포함하는 비율 데이터. |
| `eps` | object | float32 | 주당 순이익은 정수 또는 소수. float32가 안전. |
| `bps` | object | int32 | 주당 순자산은 정수. |
| `d250_hgpr` | object | int32 | 가격 데이터. |
| `d250_hgpr_date` | datetime64[ns] | datetime64[ns] | 최적 타입. 날짜 데이터이므로 타입 유지. |
| `d250_hgpr_vrss_prpr_rate` | object | float32 | 소수점을 포함하는 비율 데이터. |
| `d250_lwpr` | object | int32 | 가격 데이터. |
| `d250_lwpr_date` | datetime64[ns] | datetime64[ns] | 최적 타입. 날짜 데이터이므로 타입 유지. |
| `d250_lwpr_vrss_prpr_rate` | object | float32 | 소수점을 포함하는 비율 데이터. |
| `stck_dryy_hgpr` | object | int32 | 가격 데이터. |
| `dryy_hgpr_vrss_prpr_rate` | object | float32 | 소수점을 포함하는 비율 데이터. |
| `dryy_hgpr_date` | datetime64[ns] | datetime64[ns] | 최적 타입. 날짜 데이터이므로 타입 유지. |
| `stck_dryy_lwpr` | object | int32 | 가격 데이터. |
| `dryy_lwpr_vrss_prpr_rate` | object | float32 | 소수점을 포함하는 비율 데이터. |
| `dryy_lwpr_date` | datetime64[ns] | datetime64[ns] | 최적 타입. 날짜 데이터이므로 타입 유지. |
| `w52_hgpr` | object | int32 | 가격 데이터. |
| `w52_hgpr_vrss_prpr_ctrt` | object | float32 | 소수점을 포함하는 비율 데이터. |
| `w52_hgpr_date` | datetime64[ns] | datetime64[ns] | 최적 타입. 날짜 데이터이므로 타입 유지. |
| `w52_lwpr` | object | int32 | 가격 데이터. |
| `w52_lwpr_vrss_prpr_ctrt` | object | float32 | 소수점을 포함하는 비율 데이터. |
| `w52_lwpr_date` | datetime64[ns] | datetime64[ns] | 최적 타입. 날짜 데이터이므로 타입 유지. |
| `whol_loan_rmnd_rate` | object | float32 | 소수점을 포함하는 비율 데이터. |
| `stck_shrn_iscd` | object | category | `ticker`와 동일. category가 효율적. |
| `fcam_cnnm` | category | int32 | 액면가는 정수형 데이터이므로 int32로 변환 필요. |
| `cpfn_cnnm` | object | object | '억' 등 단위 문자열 제거 및 int64 변환 전처리 필요. |
| `frgn_hldn_qty` | object | int64 | 보유 수량은 큰 정수. |
| `last_ssts_cntg_qty` | object | int64 | 이행 수량은 큰 정수. |

In [15]:
cols_to_check = []

# 1. 모든 컬럼을 숫자 타입으로 먼저 변환
df[cols_to_check] = df[cols_to_check].apply(pd.to_numeric, errors='coerce')


# 2. 각 컬럼별로 정수 형태 데이터의 비율을 계산하는 함수를 적용
def get_integer_like_ratio(series: pd.Series) -> float:
  """
  Series의 유효한 값(NaN 제외) 중, 정수 형태인 값의 비율을 계산합니다.
  """
  # NaN 값을 제외한 데이터만 추출
  series_without_na = series.dropna()
  
  # 유효한 데이터가 없는 경우, 1.0 (100%) 반환
  if series_without_na.empty:
    return 1.0
    
  # (정수 형태인 값의 개수) / (전체 유효 데이터의 개수)
  integer_like_count = (series_without_na % 1 == 0).sum()
  total_valid_count = len(series_without_na)
  
  return integer_like_count / total_valid_count

# --- 결과 확인 ---
integer_like_ratio_result = df[cols_to_check].apply(get_integer_like_ratio)
print(integer_like_ratio_result.round(2))

Series([], dtype: float64)


In [16]:
# df.astype()에 사용하여 타입을 변경하기 위한 맵 (int64, float64 적용)
# 이미 적정 타입인 컬럼은 제외됨
dtype_map = {
  'category': ['ticker', 'bstp_kor_isnm', 'stck_shrn_iscd'],
  'bool': ['crdt_able_yn', 'elw_pblc_yn'],
  'Int64': [
    'stck_prpr', 
    'prdy_vrss', 
    'stck_oprc', 
    'stck_hgpr', 
    'stck_lwpr', 
    'stck_mxpr', 
    'stck_llam', 
    'stck_sdpr', 
    'pvt_scnd_dmrs_prc', 
    'pvt_frst_dmrs_prc', 
    'pvt_pont_val', 
    'pvt_frst_dmsp_prc', 
    'pvt_scnd_dmsp_prc', 
    'dmrs_val', 
    'dmsp_val', 
    'rstc_wdth_prc', 
    'stck_fcam', 
    'stck_sspr', 
    'aspr_unit', 
    'bps', 
    'd250_hgpr', 
    'd250_lwpr', 
    'stck_dryy_hgpr', 
    'stck_dryy_lwpr', 
    'w52_hgpr', 
    'w52_lwpr', 
    'fcam_cnnm',
    'acml_tr_pbmn', 
    'acml_vol', 
    'frgn_ntby_qty', 
    'pgtr_ntby_qty', 
    'cpfn', 
    'lstn_stcn', 
    'hts_avls', 
    'frgn_hldn_qty', 
    'last_ssts_cntg_qty'
  ],
  'float64': [
    'marg_rate', 
    'prdy_ctrt', 
    'prdy_vrss_vol_rate', 
    'wghn_avrg_stck_prc', 
    'hts_frgn_ehrt', 
    'per', 
    'pbr', 
    'vol_tnrt', 
    'eps', 
    'd250_hgpr_vrss_prpr_rate', 
    'd250_lwpr_vrss_prpr_rate', 
    'dryy_hgpr_vrss_prpr_rate', 
    'dryy_lwpr_vrss_prpr_rate', 
    'w52_hgpr_vrss_prpr_ctrt', 
    'w52_lwpr_vrss_prpr_ctrt', 
    'whol_loan_rmnd_rate'
  ]
}

for dtype, cols in dtype_map.items():
  # DataFrame에 실제 존재하는 컬럼만 필터링
  valid_cols = [col for col in cols if col in final_df.columns]
  if not valid_cols:
    continue

  try:
    if dtype == 'Int64':
      for col in valid_cols:
        numeric_col = pd.to_numeric(final_df[col], errors='coerce')
        if not numeric_col.isnull().all():
          final_df[col] = numeric_col.astype('Int64')
    
    elif dtype == 'float64':
      final_df[valid_cols] = final_df[valid_cols].apply(pd.to_numeric, errors='coerce')

    elif dtype == 'datetime64':
      for col in valid_cols:
        final_df[col] = pd.to_datetime(final_df[col], errors='coerce')
    
    else: # 'category' 등 .astype()으로 처리 가능한 나머지 타입
      final_df[valid_cols] = final_df[valid_cols].astype(dtype)

  except (ValueError, TypeError) as e:
    print(f"⚠️ 경고: 컬럼 {valid_cols}을(를) '{dtype}'으로 변환 중 오류. 건너뜁니다. (에러: {e})")


In [17]:
final_df.head()

,ticker,iscd_stat_cls_code,marg_rate,new_hgpr_lwpr_cls_code,bstp_kor_isnm,crdt_able_yn,grmn_rate_cls_code,elw_pblc_yn,stck_prpr,prdy_vrss,prdy_vrss_sign,prdy_ctrt,acml_tr_pbmn,acml_vol,prdy_vrss_vol_rate,stck_oprc,stck_hgpr,stck_lwpr,stck_mxpr,stck_llam,stck_sdpr,wghn_avrg_stck_prc,hts_frgn_ehrt,frgn_ntby_qty,pgtr_ntby_qty,pvt_scnd_dmrs_prc,pvt_frst_dmrs_prc,pvt_pont_val,pvt_frst_dmsp_prc,pvt_scnd_dmsp_prc,dmrs_val,dmsp_val,cpfn,rstc_wdth_prc,stck_fcam,stck_sspr,aspr_unit,lstn_stcn,hts_avls,per,pbr,vol_tnrt,eps,bps,d250_hgpr,d250_hgpr_date,d250_hgpr_vrss_prpr_rate,d250_lwpr,d250_lwpr_date,d250_lwpr_vrss_prpr_rate,stck_dryy_hgpr,dryy_hgpr_vrss_prpr_rate,dryy_hgpr_date,stck_dryy_lwpr,dryy_lwpr_vrss_prpr_rate,dryy_lwpr_date,w52_hgpr,w52_hgpr_vrss_prpr_ctrt,w52_hgpr_date,w52_lwpr,w52_lwpr_vrss_prpr_ctrt,w52_lwpr_date,whol_loan_rmnd_rate,stck_shrn_iscd,fcam_cnnm,cpfn_cnnm,frgn_hldn_qty,last_ssts_cntg_qty
0,005930,55,20.0,신고가,전기·전자,True,40,True,79700,-800,5,-0.99,1672547819584,20898386,81.57,81100,81200,79600,104600,56400,80500,80032.73,51.11,-537726,938828,81766,81132,79866,79232,77966,81450,79550,7780,24100,100,61180,100,5919637922,4717951,16.10,1.38,0.35,4950.0,57930,81200,2025-09-19,-1.85,49900,2024-11-14,59.72,81200,-1.85,2025-09-19,50800,56.89,2025-02-03,81200,-1.85,2025-09-19,49900,59.72,2024-11-14,0.17,005930,100,"7,780 억",3025406868,489779
1,000660,55,20.0,신고가,전기·전자,True,40,True,353000,<NA>,3,0.00,1546050743059,4385543,96.89,351500,360000,348000,458500,247500,353000,352559.79,56.08,-982503,-357729,369333,361166,348833,340666,328333,365250,344750,36577,105500,5000,254160,500,728002365,2569848,12.99,3.38,0.60,27182.0,104567,360000,2025-09-19,-1.94,144700,2024-09-19,143.95,360000,-1.94,2025-09-19,162700,116.96,2025-04-09,360000,-1.94,2025-09-19,144700,143.95,2024-09-19,0.16,000660,<NA>,"36,576 억",408260104,84763
2,373220,55,20.0,NaN,전기·전자,True,40,True,349500,-5000,5,-1.41,64980756750,185906,106.97,351500,354500,347500,460500,248500,354500,349535.95,4.09,-23961,-36176,359500,357000,352000,349500,344500,358250,350750,1170,106000,500,269420,500,234000000,817830,-80.27,3.87,0.08,-4354.0,90240,444000,2024-10-08,-21.28,266000,2025-05-23,31.39,403000,-13.28,2025-07-31,266000,31.39,2025-05-23,444000,-21.28,2024-10-08,266000,31.39,2025-05-23,0.11,373220,500,"1,170 억",9580313,45469
3,207940,55,20.0,NaN,제약,True,40,True,1024000,-5000,5,-0.49,93367438000,91201,177.89,1023000,1030000,1020000,1337000,721000,1029000,1023779.40,12.86,5913,7932,1037666,1033332,1025666,1021332,1013666,1035500,1023500,1779,308000,2500,802620,1000,71174000,728822,67.28,6.68,0.13,15221.0,153212,1209000,2025-02-14,-15.30,907000,2024-11-18,12.90,1209000,-15.30,2025-02-14,924000,10.82,2025-01-02,1209000,-15.30,2025-02-14,907000,12.90,2024-11-18,0.09,207940,<NA>,"1,779 억",9150802,1706
4,012450,55,20.0,NaN,운송장비·부품,True,40,True,1022000,-1000,5,-0.10,372660454884,363904,173.56,1010000,1041000,1005000,1329000,717000,1023000,1024102.56,44.77,3680,-2295,1077666,1050332,1024666,997332,971666,1037500,984500,2703,306000,5000,654720,1000,51563401,526978,22.13,9.44,0.71,46183.0,108270,1052000,2025-09-18,-2.85,265870,2024-12-10,284.40,1052000,-2.85,2025-09-18,325445,214.03,2025-01-02,1052000,-2.85,2025-09-18,270000,278.52,2024-12-10,0.34,012450,<NA>,"2,703 억",23082752,5937


In [18]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 68 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   ticker                    200 non-null    category      
 1   iscd_stat_cls_code        200 non-null    category      
 2   marg_rate                 200 non-null    float64       
 3   new_hgpr_lwpr_cls_code    200 non-null    category      
 4   bstp_kor_isnm             200 non-null    category      
 5   crdt_able_yn              200 non-null    bool          
 6   grmn_rate_cls_code        189 non-null    category      
 7   elw_pblc_yn               200 non-null    bool          
 8   stck_prpr                 200 non-null    Int64         
 9   prdy_vrss                 192 non-null    Int64         
 10  prdy_vrss_sign            200 non-null    category      
 11  prdy_ctrt                 200 non-null    float64       
 12  acml_tr_pbmn          